# Notebook 02: Regression Model

**Student Name:** Sean McManus

**Dataset:** Global_Space_Exploration_Dataset.csv

**Target Variable:** success_rate

**Checkpoint 3 Due:** Feb 15

---

## Rules & Integrity

1. **NO AI TOOLS**: You may **NOT** use ChatGPT, Claude, Gemini, GitHub Copilot, or any other AI assistant to generate code for this assignment. The goal is to build *your* fundamental skills. If you rely on AI now, the advanced topics later will be impossible.

2. **Study Groups Encouraged**: You **ARE** encouraged to discuss ideas, share approaches, and explain concepts to your study group peers. Teaching others is the best way to learn! However, the code you submit must be **your own work**.

3. **Use Your Resources**: You are free to use Google, StackOverflow, Pandas/Scikit-learn documentation, and your class notes.

4. **Comment Your Code**: Include comments explaining *why* you're doing what you're doing. I want to see your thought process.

5. **Resubmission**: You may submit this assignment multiple times for feedback before the checkpoint deadline.

---

## Important: Written Reflections

Throughout this notebook, you'll be asked to interpret results, justify decisions, and explain your reasoning. **These written reflections are a critical part of your grade.**

Good data scientists don't just run code—they communicate their findings clearly. Take time to write thoughtful, complete responses to all reflection prompts. This demonstrates your understanding and prepares you for real-world stakeholder communication.

---

## Setup

In [58]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn - preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

# Sklearn - models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Sklearn - evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Model saving
import joblib

# Settings
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries imported successfully!")

Libraries imported successfully!


## Section 1: Load Processed Data

In [59]:
# Load cleaned data from previous notebook
df = pd.read_csv("Global_Space_Exploration_Dataset.csv")

print(f"Data shape: {df.shape}")
df.head()

Data shape: (3000, 28)


,Year,Budget (in Billion $),Success Rate (%),Duration (in Days),Num_Collaborators,Tech_Maturity,Budget_Per_Day,Mission Type_Unmanned,Technology Used_Nuclear Propulsion,Technology Used_Reusable Rocket,...,Country_UAE,Country_UK,Country_USA,Satellite Type_Navigation,Satellite Type_Research,Satellite Type_Spy,Satellite Type_Weather,Environmental Impact_Low,Environmental Impact_Medium,Tech_Budget_Power
0,2008,16.20,90.0,112,3,3.0,0.143363,False,True,False,...,False,False,False,False,False,False,False,False,True,48.60
1,2018,29.04,99.0,236,2,2.0,0.122532,False,False,False,...,False,False,False,False,False,False,False,False,False,58.08
2,2013,28.73,54.0,238,3,1.0,0.120209,False,False,False,...,False,False,False,False,False,False,False,False,True,28.73
3,2010,37.27,58.0,186,1,1.0,0.199305,True,False,False,...,True,False,False,False,False,True,False,True,False,37.27
4,2006,18.95,91.0,277,3,2.0,0.068165,False,False,False,...,False,False,False,False,False,False,True,False,True,37.90


In [60]:
print(df.columns.tolist()) #Quick check to confirm name


['Year', 'Budget (in Billion $)', 'Success Rate (%)', 'Duration (in Days)', 'Num_Collaborators', 'Tech_Maturity', 'Budget_Per_Day', 'Mission Type_Unmanned', 'Technology Used_Nuclear Propulsion', 'Technology Used_Reusable Rocket', 'Technology Used_Solar Propulsion', 'Technology Used_Traditional Rocket', 'Country_France', 'Country_Germany', 'Country_India', 'Country_Israel', 'Country_Japan', 'Country_Russia', 'Country_UAE', 'Country_UK', 'Country_USA', 'Satellite Type_Navigation', 'Satellite Type_Research', 'Satellite Type_Spy', 'Satellite Type_Weather', 'Environmental Impact_Low', 'Environmental Impact_Medium', 'Tech_Budget_Power']


In [61]:
# Define your target variable
TARGET = 'Success Rate (%)'  # <-- UPDATE THIS!

print(f"Target: {TARGET}")
print(f"\nTarget statistics:")
print(df[TARGET].describe())

# Store target range for later interpretation
target_range = df[TARGET].max() - df[TARGET].min()
target_std = df[TARGET].std()
print(f"\nTarget range: {target_range:,.2f}")
print(f"Target std: {target_std:,.2f}")

Target: Success Rate (%)

Target statistics:
count    3000.000000
mean       74.985000
std        14.945252
min        50.000000
25%        62.000000
50%        75.000000
75%        88.000000
max       100.000000
Name: Success Rate (%), dtype: float64

Target range: 50.00
Target std: 14.95


## Section 2: Prepare Features and Target

In [62]:
# Separate features (X) and target (y)
X = df.drop(columns=[TARGET])
y = df[TARGET]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nAll features ({len(X.columns)}):")
print(X.columns.tolist())

Features shape: (3000, 27)
Target shape: (3000,)

All features (27):
['Year', 'Budget (in Billion $)', 'Duration (in Days)', 'Num_Collaborators', 'Tech_Maturity', 'Budget_Per_Day', 'Mission Type_Unmanned', 'Technology Used_Nuclear Propulsion', 'Technology Used_Reusable Rocket', 'Technology Used_Solar Propulsion', 'Technology Used_Traditional Rocket', 'Country_France', 'Country_Germany', 'Country_India', 'Country_Israel', 'Country_Japan', 'Country_Russia', 'Country_UAE', 'Country_UK', 'Country_USA', 'Satellite Type_Navigation', 'Satellite Type_Research', 'Satellite Type_Spy', 'Satellite Type_Weather', 'Environmental Impact_Low', 'Environmental Impact_Medium', 'Tech_Budget_Power']


In [63]:
# Check for any non-numeric columns that need to be handled
non_numeric = X.select_dtypes(include=['object', 'category']).columns.tolist()
if non_numeric:
    print(f"Warning: Non-numeric columns found: {non_numeric}")
    print("You need to encode these or go back to Notebook 01!")
else:
    print("All features are numeric. Ready to proceed!")

All features are numeric. Ready to proceed!


## Section 3: Train-Test Split

Split your data into training and test sets. The training set is used to train the model, and the test set is used to evaluate how well it generalizes to unseen data.

In [64]:
# TODO: Split your data into training and test sets
# 
# Requirements:
# - Use an 80/20 split (test_size=0.2)
# - Set random_state=42 for reproducibility # the best number
# - Store results in: X_train, X_test, y_train, y_test

# Hint: Use train_test_split(X, y, ...)
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)
# lest check this worked
print(f"Training set: {X_train.shape[0]} missions")
print(f"Testing set: {X_test.shape[0]} missions")
print(f"Total features: {X_train.shape[1]}")

#thanks David for that check


Training set: 2400 missions
Testing set: 600 missions
Total features: 27


In [65]:
# Verify your split (run this cell to check)
print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.0f}%)")

Training set: 2,400 samples (80%)
Test set: 600 samples (20%)


## Section 4: Feature Scaling

Many ML algorithms perform better when features are on similar scales. StandardScaler transforms features to have mean=0 and std=1.

**Important:** Fit the scaler on training data only, then transform both train and test. This prevents data leakage.

In [66]:
# TODO: Scale your features using StandardScaler
#
# Steps:
# 1. Create a StandardScaler instance #### Instance???  = object = the actual scaler i will use
# 2. Fit the scaler on X_train and transform X_train (use fit_transform)  ### ok fit_transform will calculate mean and standard dev of X-train
# 3. Transform X_test using the same scaler (use transform only - NOT fit_transform!)
# 4. Store results in: X_train_scaled, X_test_scaled
#
# Why fit only on train? To prevent "data leakage" - test data should be truly unseen.
# Its Scaler // not scalar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)    ##PUT CORRECT SET IN SCALER FIT TRANSFORM***Not testing data X_TRAIN IMportant

#!!from step 3!!! dont use fit_transform for test data 
X_test_scaled = scaler.transform(X_test)



In [67]:
# Convert back to DataFrame for easier viewing (optional but helpful)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

# Verify scaling worked
print("Features scaled successfully!")
print(f"\nScaled feature means (should be ~0): {X_train_scaled.mean().mean():.4f}")
print(f"Scaled feature stds (should be ~1): {X_train_scaled.std().mean():.4f}")

Features scaled successfully!

Scaled feature means (should be ~0): -0.0000
Scaled feature stds (should be ~1): 1.0002


## Section 5: Baseline Model

Start with a simple Linear Regression to establish a baseline performance. This gives us a reference point for improvement.

In [68]:
# This helper function evaluates any model - you'll use it throughout this notebook
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Train model and return evaluation metrics."""
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    results = {
        'Model': model_name,
        'Train R2': r2_score(y_train, y_train_pred),
        'Test R2': r2_score(y_test, y_test_pred),
        'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        'Train MAE': mean_absolute_error(y_train, y_train_pred),
        'Test MAE': mean_absolute_error(y_test, y_test_pred)
    }
    
    return results, model, y_test_pred

In [69]:
# TODO: Create and evaluate a baseline Linear Regression model
#
# Steps:
# 1. Create a LinearRegression() model instance
# 2. Use the evaluate_model() function to train and evaluate it
# 3. Store the results  ### DICTIONARY
#
# The evaluate_model function returns: (results_dict, trained_model, predictions)

# YOUR CODE HERE:
# baseline_model = ...
# baseline_results, baseline_trained, baseline_preds = evaluate_model(...)
baseline_model = LinearRegression()
#scaled data is what is necissary for the dictinary
baseline_results, baseline_trained, baseline_preds = evaluate_model(
    model = baseline_model,
    X_train = X_train_scaled,  ##USE SCALED DATA
    X_test = X_test_scaled,
    y_train = y_train,
    y_test = y_test,
    model_name = "Baseline Spaceline Linear Regression Model"
)
#/// dictionary classification is case sensitive frm now on!! (*model mistake*)

In [70]:
# Display baseline results
print("=" * 50)
print("BASELINE MODEL: Linear Regression")
print("=" * 50)
print(f"Train R²: {baseline_results['Train R2']:.4f}")
print(f"Test R²:  {baseline_results['Test R2']:.4f}")
print(f"Test RMSE: {baseline_results['Test RMSE']:,.2f}")
print(f"Test MAE: {baseline_results['Test MAE']:,.2f}")

# Context for RMSE
print(f"\n--- RMSE in Context ---")
print(f"RMSE as % of target range: {baseline_results['Test RMSE']/target_range*100:.1f}%")
print(f"RMSE as % of target std: {baseline_results['Test RMSE']/target_std*100:.1f}%")

BASELINE MODEL: Linear Regression
Train R²: 0.0076
Test R²:  -0.0096
Test RMSE: 15.28
Test MAE: 13.34

--- RMSE in Context ---
RMSE as % of target range: 30.6%
RMSE as % of target std: 102.2%


In [71]:
print(pd.Series(baseline_results)) ## SUber neat way i can do this n the future

Model         Baseline Spaceline Linear Regression Model
Train R2                                        0.007616
Test R2                                        -0.009644
Train RMSE                                     14.819956
Test RMSE                                      15.277089
Train MAE                                      12.879073
Test MAE                                       13.338825
dtype: object


### Baseline Interpretation

*Analyze your baseline results:*

**Understanding your metrics:**
- **R² (0 to 1)**: How much variance in target is explained by features. 0.7+ is often considered good. # :/
- **RMSE**: Average prediction error in the same units as your target. Lower is better.
- **RMSE as % of range**: Helps you understand if errors are big or small relative to your data.

**Questions to answer:**
- What does your R² score tell you about how well features explain the target?
- Is there a big gap between train and test R²? (Gap > 0.1 could indicate overfitting)
- Is your RMSE reasonable? (e.g., if predicting house prices, is a $20K error acceptable?)

**Your interpretation:**

-Ok so test R^2 of 0.0096 (about 0.01 )... thats worse than guessing the success rate of each mission.. and the train r^2 is 0.0076. So the features are explaining less that 1% of the variations in missions and success rates, and can be considered "blind".
-There is no big gap. the train and test R^2 are both around zero. So the mode lisnt overfitting, but since these values are so low its Underfitting.. this is probably not be the model I should use.
- the RMSE is 15.28 and my target variable is success rate of a dadgum Space mission worth billions of dollars, which is a percentage range from 0-100. 15% is a HUGE issue for a space mission. So not reasonable, definitely not for the kind of predictions we need to make; Expensive Predictions. Maybe even life dependant missions. 

I might have to trim this down. I have so many columns. i think it might have too many features. especially the One-hot encoded ones. if my missions are either (FAILURE) or (Success) theres not much middle ground to navigate, maybe making a regression model difficult for this situation.

## Section 6: Model Iteration

Try at least 2-3 different models to see if you can improve on the baseline.

lets go with ridge first for the noisy features.

In [72]:
# Store all results for comparison
all_results = [baseline_results]

# Dictionary to store trained models
trained_models = {
    'Linear Regression (Baseline)': baseline_trained
}

In [73]:
"""
ridge_results, ridge_trained, ridge_preds = evaluate_model(
    Ridge(alpha=1.0), 
    X_train_scaled, X_test_scaled, y_train, y_test, 
    "Ridge Regression"
)
print(pd.Series(ridge_results))
"""

'\nridge_results, ridge_trained, ridge_preds = evaluate_model(\n    Ridge(alpha=1.0), \n    X_train_scaled, X_test_scaled, y_train, y_test, \n    "Ridge Regression"\n)\nprint(pd.Series(ridge_results))\n'

### Model 2: Ridge Regression

Ridge adds L2 regularization to prevent overfitting by penalizing large coefficients.

In [74]:
# TODO: Create and evaluate a Ridge Regression model
#
# Create a Ridge model with alpha=1.0
# Use evaluate_model() to train and evaluate
# Add results to all_results list
# Add trained model to trained_models dict

ridge_model = Ridge(alpha=1.0)
ridge_results, ridge_trained, ridge_preds = evaluate_model(
    model = ridge_model,
    X_train = X_train_scaled,
    X_test = X_test_scaled,
    y_train = y_train,
    y_test = y_test,
    model_name = "Ridg-E-Mod Ridge Regression Model"

)
#have to add the trained model results to the model list in the secsion before this one.
all_results.append(ridge_results)
trained_models["Ridge Regression"] = ridge_trained

# Print results
print(f"Ridge Regression - Test R²: {ridge_results['Test R2']:.4f}, Test RMSE: {ridge_results['Test RMSE']:,.2f}")

Ridge Regression - Test R²: -0.0091, Test RMSE: 15.27


In [75]:
# oh lord.. lets get it all
print(pd.Series(ridge_results))

Model         Ridg-E-Mod Ridge Regression Model
Train R2                               0.007629
Test R2                               -0.009128
Train RMSE                            14.819859
Test RMSE                             15.273187
Train MAE                             12.879627
Test MAE                               13.33516
dtype: object


### Model 3: Lasso Regression

Lasso adds L1 regularization, which can zero out unimportant features (automatic feature selection).

In [79]:
# TODO: Create and evaluate a Lasso Regression model
#
# Create a Lasso model with alpha=0.1
# Use evaluate_model() to train and evaluate
# Add results to all_results and trained_models

lasso_model = Lasso(alpha=0.1)  ### 0.1 // not 1.0 that was for the ridge

lasso_results, lasso_trained, lasso_preds = evaluate_model(
    model = lasso_model,
    X_train = X_train_scaled,
    X_test = X_test_scaled,
    y_train = y_train,
    y_test = y_test,
    model_name = "Lasso Regression Yee-Haw"

)

all_results.append(lasso_results)
trained_models["Lasso Regression"] = lasso_trained
# Print results
print(f"Lasso Regression - Test R²: {lasso_results['Test R2']:.4f}, Test RMSE: {lasso_results['Test RMSE']:,.2f}")

# Show which features Lasso kept (non-zero coefficients)
lasso_coefs = pd.Series(lasso_trained.coef_, index=X.columns)
kept_features = lasso_coefs[lasso_coefs != 0]
print(f"\nLasso kept {len(kept_features)} of {len(X.columns)} features")
print("Kept features:")
print(kept_features.sort_values())

Lasso Regression - Test R²: -0.0039, Test RMSE: 15.23

Lasso kept 18 of 27 features
Kept features:
Technology Used_Nuclear Propulsion   -0.230719
Budget_Per_Day                       -0.165288
Satellite Type_Spy                   -0.161222
Mission Type_Unmanned                -0.128107
Duration (in Days)                   -0.099899
Country_USA                          -0.089839
Country_Israel                       -0.026457
Country_Russia                        0.040910
Tech_Budget_Power                     0.071931
Num_Collaborators                     0.114028
Year                                  0.155417
Satellite Type_Research               0.157598
Environmental Impact_Low              0.158041
Satellite Type_Navigation             0.160926
Country_India                         0.225194
Country_Germany                       0.245694
Country_France                        0.246360
Technology Used_Reusable Rocket       0.534895
dtype: float64


 YOOOOO? Reuseable Rocket looking really good really putting my hopes on the decision tree

In [80]:
print(pd.Series(lasso_results))

Model         Lasso Regression Yee-Haw
Train R2                      0.006238
Test R2                      -0.003939
Train RMSE                    14.83024
Test RMSE                    15.233868
Train MAE                    12.895708
Test MAE                      13.31361
dtype: object


### Model 4: Decision Tree

In [ ]:
# TODO: Create and evaluate a Decision Tree model
#
# Create a DecisionTreeRegressor with max_depth=10 and random_state=42
# Use evaluate_model() to train and evaluate
# Add results to all_results and trained_models

tree_model = DecisionTreeRegressor(max_depth=10, random_state=42) #gotta mage sure to get the case sensitive // could not tell where i went wrong

tree_results, tree_trained, tree_preds = evaluate_model(
    model = tree_model,
    X_train = X_train_scaled,
    X_test = X_test_scaled,
    y_train = y_train,
    y_test = y_test,
    model_name = "Decission-Maker Regression Arts: 「Yggdrasil」"  
)
all_results.append(tree_results)
trained_models["Decission-Maker Regression Arts: 「Yggdrasil」"] = tree_trained
# Print results /// woops gotta change this to match the model name in the dictinary
print(f"Decision Tree - Test R²: {tree_results['Test R2']:.4f}, Test RMSE: {tree_results['Test RMSE']:,.2f}")



Decision Tree - Test R²: -0.4060, Test RMSE: 18.03


In [ ]:
from sklearn.tree import export_text #did a little googleing to see if i can get something a little show-y
tree_rules = export_text(tree_trained, feature_names=list(X.columns), max_depth=2)
print("\nTop levels of the Decision Tree:")
print(tree_rules)


Top levels of the Decision Tree:
|--- Duration (in Days) <= -1.32
|   |--- Budget (in Billion $) <= -0.32
|   |   |--- Country_UK <= 1.28
|   |   |   |--- truncated branch of depth 8
|   |   |--- Country_UK >  1.28
|   |   |   |--- truncated branch of depth 7
|   |--- Budget (in Billion $) >  -0.32
|   |   |--- Budget (in Billion $) <= 1.37
|   |   |   |--- truncated branch of depth 8
|   |   |--- Budget (in Billion $) >  1.37
|   |   |   |--- truncated branch of depth 8
|--- Duration (in Days) >  -1.32
|   |--- Duration (in Days) <= 0.48
|   |   |--- Tech_Budget_Power <= 2.06
|   |   |   |--- truncated branch of depth 8
|   |   |--- Tech_Budget_Power >  2.06
|   |   |   |--- truncated branch of depth 8
|   |--- Duration (in Days) >  0.48
|   |   |--- Year <= -1.48
|   |   |   |--- truncated branch of depth 8
|   |   |--- Year >  -1.48
|   |   |   |--- truncated branch of depth 8



In [81]:
print(pd.Series(tree_results))

Model         Decission-Maker Regression Arts: 「Yggdrasil」
Train R2                                          0.379376
Test R2                                          -0.405995
Train RMSE                                       11.719828
Test RMSE                                        18.028037
Train MAE                                         8.942275
Test MAE                                         15.004797
dtype: object


ok so, theres actuall a large gap between train and test  R^2. so a case of overfitting with this model. so that means finally the most optimal model i should do is Random Forest. (you knew this all along Abishek you sly dog! my heart was breaking out here)

### Model 5: Random Forest

In [ ]:
# TODO: Create and evaluate a Random Forest model
#
# Create a RandomForestRegressor with: /// TIME FOR SOME ENSEMBLE LEARNING BAYBEEEEEEE
#   - n_estimators=100
#   - max_depth=10
#   - random_state=42
#   - n_jobs=-1 (use all CPU cores)  If I don't show up to class, let the class know my crappy laptop cooked off like an anti-tank mine 
#
# Use evaluate_model() to train and evaluate
# Add results to all_results and trained_models
#### set up the random forest parameters first, Its not like the other models 
rf_model = RandomForestRegressor(   
    n_estimators=100, 
    max_depth=10, 
    random_state=42, 
    n_jobs=-1
)



# Print results
print(f"Random Forest - Test R²: {rf_results['Test R2']:.4f}, Test RMSE: {rf_results['Test RMSE']:,.2f}")

### Add Your Own Model (Optional)

*Feel free to try additional models or tune hyperparameters!*

In [ ]:
# YOUR ADDITIONAL MODEL HERE:
# Try GradientBoostingRegressor, different hyperparameters, etc.



## Section 7: Cross-Validation (More Robust Evaluation)

Cross-validation gives us a more reliable estimate of model performance by testing on multiple different train/test splits.

In [ ]:
# Perform 5-fold cross-validation on top models
print("5-Fold Cross-Validation Results:")
print("=" * 50)

cv_results = []

models_to_cv = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
}

for name, model in models_to_cv.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    cv_results.append({
        'Model': name,
        'CV Mean R²': scores.mean(),
        'CV Std R²': scores.std()
    })
    print(f"{name}: R² = {scores.mean():.4f} (+/- {scores.std()*2:.4f})")

cv_df = pd.DataFrame(cv_results)

**Why Cross-Validation Matters:**
- A model that performs well on one train/test split might just be lucky
- CV tests on 5 different splits, giving us confidence in the results
- Lower standard deviation = more consistent/reliable model

## Section 8: Model Comparison

In [ ]:
# Create comparison DataFrame
results_df = pd.DataFrame(all_results)
results_df = results_df.round(4)
results_df = results_df.sort_values('Test R2', ascending=False)

print("Model Comparison:")
results_df

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² Comparison
models = results_df['Model']
x = np.arange(len(models))
width = 0.35

axes[0].bar(x - width/2, results_df['Train R2'], width, label='Train R²', alpha=0.8)
axes[0].bar(x + width/2, results_df['Test R2'], width, label='Test R²', alpha=0.8)
axes[0].set_ylabel('R² Score')
axes[0].set_title('R² Comparison Across Models')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].legend()
axes[0].set_ylim(0, 1.1)
axes[0].axhline(y=0.7, color='green', linestyle='--', alpha=0.5, label='Good threshold')

# RMSE Comparison
axes[1].bar(x - width/2, results_df['Train RMSE'], width, label='Train RMSE', alpha=0.8)
axes[1].bar(x + width/2, results_df['Test RMSE'], width, label='Test RMSE', alpha=0.8)
axes[1].set_ylabel('RMSE')
axes[1].set_title('RMSE Comparison Across Models')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models, rotation=45, ha='right')
axes[1].legend()

plt.tight_layout()
plt.show()

### Model Iteration Reflection

*Before selecting your best model, reflect on what you learned:*

**Questions to answer:**
- Which model improved most over the baseline?
- Did any models show signs of overfitting (big train vs test gap)?
- Did regularization (Ridge/Lasso) help or hurt performance?
- Did tree-based models (Decision Tree, Random Forest) work better than linear models?

**Your reflection:**

[Write your reflection here]

## Section 9: Feature Importance & Selection

**Important:** Your final model should use only **4-8 features**. This section helps you identify which features matter most.

In [ ]:
# Get feature importance from Random Forest (works well for this)
rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_trained.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance (Random Forest):")
print(rf_importance)

# Visualize
plt.figure(figsize=(10, 8))
plt.barh(rf_importance['Feature'][::-1], rf_importance['Importance'][::-1])
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)')
plt.tight_layout()
plt.show()

In [ ]:
# Also check correlations with target
correlations = X_train.corrwith(y_train).abs().sort_values(ascending=False)
print("Absolute Correlations with Target:")
print(correlations)

In [ ]:
# TODO: Select your top features (4-8 features)
#
# Based on the importance analysis above, choose your best features.
# Consider both Random Forest importance AND correlations.
# Also think about what makes sense from a domain perspective.

SELECTED_FEATURES = [
    # YOUR FEATURES HERE - add 4-8 feature names:
    # 'feature1',
    # 'feature2',
    # etc.
]

# Fallback if you haven't selected yet
if len(SELECTED_FEATURES) == 0:
    SELECTED_FEATURES = rf_importance['Feature'].head(6).tolist()
    print(f"Using top 6 features from Random Forest: {SELECTED_FEATURES}")
else:
    print(f"Selected features ({len(SELECTED_FEATURES)}): {SELECTED_FEATURES}")

In [ ]:
# Retrain with selected features only
X_train_selected = X_train_scaled[SELECTED_FEATURES]
X_test_selected = X_test_scaled[SELECTED_FEATURES]

print(f"Training with {len(SELECTED_FEATURES)} selected features...")

# Test a few models with selected features
selected_results = []

for name, model in [('Linear Regression', LinearRegression()),
                    ('Ridge', Ridge(alpha=1.0)),
                    ('Random Forest', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))]:
    results, trained, _ = evaluate_model(model, X_train_selected, X_test_selected, y_train, y_test, name)
    selected_results.append(results)
    print(f"{name} with {len(SELECTED_FEATURES)} features - Test R²: {results['Test R2']:.4f}")

selected_df = pd.DataFrame(selected_results)

### Feature Selection Justification

**Questions to answer:**
- Which features did you select and why?
- Did performance drop significantly with fewer features?
- Do these features make sense from a domain perspective?
- These are the features users will input in your Streamlit app—are they reasonable to ask for?

**Your justification:**

[Write your justification here]

## Section 10: Best Model Selection

In [ ]:
# TODO: Choose and train your final model with selected features
#
# Based on your analysis, pick the best model type and instantiate it.
# Consider: performance, consistency, simplicity
#
# Example: final_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

# YOUR CODE HERE:
# final_model = ...



In [ ]:
# Fit and evaluate final model
final_model.fit(X_train_selected, y_train)
y_pred = final_model.predict(X_test_selected)

# Final metrics
final_r2 = r2_score(y_test, y_pred)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
final_mae = mean_absolute_error(y_test, y_pred)

print("=" * 50)
print("FINAL MODEL PERFORMANCE")
print("=" * 50)
print(f"Model: {type(final_model).__name__}")
print(f"Features: {SELECTED_FEATURES}")
print(f"\nTest R²: {final_r2:.4f}")
print(f"Test RMSE: {final_rmse:,.2f}")
print(f"Test MAE: {final_mae:,.2f}")
print(f"\nRMSE as % of target range: {final_rmse/target_range*100:.1f}%")

### Best Model Justification

*Explain why you chose this model as your best:*

**Questions to consider:**
- Why did you select this model over others?
- Is there significant overfitting (train vs test gap)?
- How does the performance compare to your baseline?
- Would a simpler model be almost as good?
- Does the RMSE represent acceptable prediction error for your problem?

**Your justification:**

[Write your justification here]

## Section 11: Best Model Analysis

In [ ]:
# Actual vs Predicted plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Actual vs Predicted ({type(final_model).__name__})')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
residuals = y_test - y_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residual distribution (should be roughly normal, centered at 0)
axes[0].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Residual (Actual - Predicted)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Residuals')

# Residuals vs Predicted (should show no pattern)
axes[1].scatter(y_pred, residuals, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title('Residuals vs Predicted Values')

plt.tight_layout()
plt.show()

print(f"Residual mean (should be ~0): {residuals.mean():.2f}")
print(f"Residual std: {residuals.std():,.2f}")

In [ ]:
# Feature importance for final model (with selected features)
if hasattr(final_model, 'feature_importances_'):
    final_importance = pd.DataFrame({
        'Feature': SELECTED_FEATURES,
        'Importance': final_model.feature_importances_
    }).sort_values('Importance', ascending=True)
    
    plt.figure(figsize=(10, 6))
    plt.barh(final_importance['Feature'], final_importance['Importance'])
    plt.xlabel('Importance')
    plt.title('Feature Importance (Final Model)')
    plt.tight_layout()
    plt.show()
elif hasattr(final_model, 'coef_'):
    final_importance = pd.DataFrame({
        'Feature': SELECTED_FEATURES,
        'Coefficient': final_model.coef_
    }).sort_values('Coefficient', key=abs, ascending=True)
    
    plt.figure(figsize=(10, 6))
    colors = ['green' if c > 0 else 'red' for c in final_importance['Coefficient']]
    plt.barh(final_importance['Feature'], final_importance['Coefficient'], color=colors)
    plt.xlabel('Coefficient')
    plt.title('Feature Coefficients (Final Model)')
    plt.tight_layout()
    plt.show()

## Section 12: Save Model and Artifacts

In [ ]:
# Create a new scaler fitted only on selected features
final_scaler = StandardScaler()
X_train_final = X_train[SELECTED_FEATURES]
final_scaler.fit(X_train_final)

# Save the best model
model_path = '../models/regression_model.pkl'
joblib.dump(final_model, model_path)
print(f"Model saved to {model_path}")

# Save the scaler (fitted on selected features only)
scaler_path = '../models/regression_scaler.pkl'
joblib.dump(final_scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

# Save feature names (the selected features for Streamlit app)
features_path = '../models/regression_features.pkl'
joblib.dump(SELECTED_FEATURES, features_path)
print(f"Features saved to {features_path}")

In [ ]:
# Verify saved model works
loaded_model = joblib.load(model_path)
loaded_scaler = joblib.load(scaler_path)
loaded_features = joblib.load(features_path)

# Test prediction
test_sample = X_test[loaded_features].iloc[[0]]
test_sample_scaled = loaded_scaler.transform(test_sample)
test_pred = loaded_model.predict(test_sample_scaled)

print(f"\nModel verification:")
print(f"Features used: {loaded_features}")
print(f"Sample input: {test_sample.values[0]}")
print(f"Predicted: {test_pred[0]:,.2f}")
print(f"Actual: {y_test.iloc[0]:,.2f}")
print(f"\nModel saved and verified successfully!")

## Section 13: Binning Strategy for Classification

**IMPORTANT:** Before you start Notebook 03, you need to send your binning strategy to Abishek on Slack for approval by **Feb 15**.

### Analyze Target Distribution

In [ ]:
# Look at target distribution to help decide binning
print(f"Target: {TARGET}")
print(f"\nDescriptive Statistics:")
print(y.describe())

print(f"\nPercentiles:")
for p in [10, 25, 33, 50, 67, 75, 90]:
    print(f"{p}th percentile: {y.quantile(p/100):,.2f}")

In [ ]:
# Visualize potential binning strategies
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram with quartile lines
axes[0].hist(y, bins=30, edgecolor='black', alpha=0.7)
for p, color, style in [(25, 'orange', '--'), (50, 'red', '-'), (75, 'orange', '--')]:
    axes[0].axvline(y.quantile(p/100), color=color, linestyle=style, linewidth=2,
                     label=f'{p}th percentile: {y.quantile(p/100):,.0f}')
axes[0].set_xlabel(TARGET)
axes[0].set_title('Distribution with Quartile Lines')
axes[0].legend()

# Box plot
axes[1].boxplot(y)
axes[1].set_ylabel(TARGET)
axes[1].set_title('Box Plot of Target')

plt.tight_layout()
plt.show()

### Proposed Binning Strategy

*Complete this section and send to Abishek on Slack by Feb 15:*

**1. How many categories will you create?**

[Your answer - e.g., 3 categories: Low, Medium, High]

**2. What are your bin thresholds?**

[Your answer - e.g., Low: < $30,000, Medium: $30,000-$60,000, High: > $60,000]

**3. Why does this binning make sense for your problem?**

[Your answer - explain the domain reasoning. Why are these meaningful categories?]

In [ ]:
# Preview your binning
def create_bins_preview(y, strategy='quartile'):
    if strategy == 'quartile':
        bins = [y.min()-1, y.quantile(0.25), y.quantile(0.75), y.max()+1]
        labels = ['Low', 'Medium', 'High']
    elif strategy == 'tertile':
        bins = [y.min()-1, y.quantile(0.33), y.quantile(0.67), y.max()+1]
        labels = ['Low', 'Medium', 'High']
    # Add your custom strategy here if needed
    
    return pd.cut(y, bins=bins, labels=labels)

# Preview with quartile binning
y_binned = create_bins_preview(y, 'quartile')
print("Preview of binned target (using quartiles):")
print(y_binned.value_counts().sort_index())
print(f"\nPercentages:")
print((y_binned.value_counts(normalize=True) * 100).round(1).sort_index())

---

## Summary

### What You Accomplished
- [ ] Loaded and prepared cleaned data
- [ ] Split data into train and test sets
- [ ] Scaled features appropriately
- [ ] Built a baseline model
- [ ] Tried multiple model types
- [ ] Performed cross-validation
- [ ] Selected top 4-8 features
- [ ] Compared and selected best model
- [ ] Analyzed model performance (residuals, feature importance)
- [ ] Saved model, scaler, and feature list
- [ ] Planned binning strategy for classification

### Key Results

**Best Model:** [Model name]

**Selected Features:** [List them]

**Test R²:** [Value]

**Test RMSE:** [Value]

**Improvement over baseline:** [Percentage or description]

---

---

## Checkpoint 3 Submission Instructions

**Congratulations!** You've completed Checkpoint 3 (Regression Model).

### Step 1: Save This Notebook
- File -> Save (or Ctrl+S / Cmd+S)

### Step 2: Send Binning Strategy to Abishek
- Message Abishek on Slack with your binning strategy
- Include: number of categories, thresholds, and justification

### Step 3: Commit to GitHub
 !!!!!!!!!!!!!! I have been opening my gitbash directy in my capstone project folder an pushing stuff. just using regular git commands was not working. I think its the same with the data and the models. I'm not sure how to get it right!!!!!!
```bash
# Stage your changes
git add notebooks/02_regression_model.ipynb
git add models/

# Commit with a meaningful message
git commit -m "Complete Checkpoint 3: Regression model with feature selection"

# Push to GitHub
git push
```

### Step 4: Submit to Canvas
1. Go to the Checkpoint 3 assignment on Canvas
2. Submit the link to your GitHub repository

---

## Next Steps

1. **Wait for binning approval** from Abishek
2. Move on to **Notebook 03: Classification Model**

---